In [ ]:
# Banka Datası, Decision Tree, Random Forest, GBM (1 tanesi yeter), NN, SVM
#Haftaya sadece Naive Bayes ve model comparison kaldı 

In [5]:
import pandas as pd
import numpy as np

bankdata = pd.read_csv("C:/Users/cakahraman/Desktop/Data Science Bootcamp/hmelq.txt")

In [6]:
bdat = bankdata.copy()
bdat = bdat.dropna()
bdat.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
153,0,18200,94727.0,136877.0,DebtCon,Mgr,15.0,0.0,0.0,168.96,2.0,26.0,36.056
154,0,21700,79240.0,96784.0,DebtCon,Mgr,5.0,0.0,0.0,64.51,6.0,24.0,38.079
155,0,34100,241931.0,36486.0,DebtCon,Mgr,1.0,0.0,2.0,196.01,3.0,50.0,42.459
156,0,8400,62989.0,76718.0,HomeImp,Mgr,3.0,0.0,2.0,131.47,0.0,22.0,29.200
157,0,17400,25859.0,43684.0,DebtCon,Mgr,16.0,1.0,0.0,95.36,1.0,17.0,27.108


In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

y = bdat["bad"]
X = bdat.drop(["bad", "reason", "job"], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108


In [5]:
#CART - DECISION TREE

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
cart = DecisionTreeClassifier()
cart_model = cart.fit(X_train,y_train)

In [14]:
cart_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
!pip install skompiler 
!pip install astor

In [16]:
from skompiler import skompile

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = cart_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9273927392739274

In [20]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       554
           1       0.60      0.48      0.53        52

    accuracy                           0.93       606
   macro avg       0.77      0.73      0.75       606
weighted avg       0.92      0.93      0.92       606



In [21]:
cart_grid = {"max_depth": range(1,10),
             "min_samples_split" : list(range(2,50))}

In [22]:
cart = DecisionTreeClassifier()
cart_cv = GridSearchCV(cart, cart_grid, cv = 10, n_jobs = -1, verbose = 2)
cart_cv_model = cart_cv.fit(X_train, y_train)

Fitting 10 folds for each of 432 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 3266 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 4313 out of 4320 | elapsed:   30.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:   30.7s finished


In [23]:
print("En iyi parametreler: " + str(cart_cv_model.best_params_))

En iyi parametreler: {'max_depth': 8, 'min_samples_split': 3}


In [24]:
cart = DecisionTreeClassifier(max_depth = 8, min_samples_split = 3)
cart_tuned = cart.fit(X_train, y_train)

In [25]:
y_pred = cart_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.9455445544554455

In [26]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.83      0.46      0.59        52

    accuracy                           0.95       606
   macro avg       0.89      0.73      0.78       606
weighted avg       0.94      0.95      0.94       606



In [ ]:
#RANDOM FOREST

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rf_model = RandomForestClassifier().fit(X_train,y_train)

C:\Users\cakahraman\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
y_pred = rf_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9488448844884488

In [31]:
rf_params = {"max_depth": [2,5,8,10],
             "max_features": [2,5,8],
             "n_estimators": [10,500,100],
             "min_samples_split" : [2,5,10]}

In [32]:
rf_model = RandomForestClassifier()

rf_cv_model = GridSearchCV(rf_model, rf_params, cv =10, n_jobs = -1, verbose =5)

In [33]:
rf_cv_model.fit(X_train,y_train)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   53.3s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 874 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  5.0min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [34]:
print("En iyi parametreler:" + str(rf_cv_model.best_params_))

En iyi parametreler:{'max_depth': 10, 'max_features': 2, 'min_samples_split': 2, 'n_estimators': 500}


In [35]:
rf_tuned = RandomForestClassifier(max_depth = 10,
                                 max_features = 2, 
                                 min_samples_split = 2,
                                 n_estimators = 500)

rf_tuned.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
y_pred = rf_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

0.9471947194719472

In [37]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       554
           1       1.00      0.38      0.56        52

    accuracy                           0.95       606
   macro avg       0.97      0.69      0.76       606
weighted avg       0.95      0.95      0.94       606



In [38]:
#GBM Gradient Boosting Machine
from sklearn.ensemble import GradientBoostingClassifier

In [39]:
gbm_model = GradientBoostingClassifier().fit(X_train, y_train)

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = gbm_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9488448844884488

In [41]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.86      0.48      0.62        52

    accuracy                           0.95       606
   macro avg       0.91      0.74      0.79       606
weighted avg       0.95      0.95      0.94       606



In [42]:
gbm_params = {"learning_rate" : [0.001, 0.01, 0.1,0.05],
             "n_estimators": [100,500,1000],
              "max_depth" : [3,5,10],
              "min_samples_split" : [2,5,10]}

In [44]:
gbm = GradientBoostingClassifier()

gbm_cv = GridSearchCV(gbm, gbm_params, cv = 10, n_jobs = -1, verbose =5)

In [45]:
gbm_cv.fit(X_train,y_train)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 874 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 12.1min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_n...None,
                                                  presort='auto

In [46]:
print("En iyi parametreler:" + str(gbm_cv.best_params_))

En iyi parametreler:{'learning_rate': 0.1, 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 1000}


In [47]:
gbm = GradientBoostingClassifier(learning_rate = 0.1,
                                max_depth =10,
                                min_samples_split = 2,
                                n_estimators = 1000)

In [48]:
gbm_tuned = gbm.fit(X_train, y_train)

In [49]:
y_pred = gbm_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.9488448844884488

In [50]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.84      0.50      0.63        52

    accuracy                           0.95       606
   macro avg       0.90      0.75      0.80       606
weighted avg       0.94      0.95      0.94       606



In [ ]:
#NN Neural Network 

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

In [14]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
X_test_scaled[0:5]

array([[ 0.66119779,  0.41857973,  0.45688065, -1.1976411 ,  1.5763105 ,
        -0.32687081,  0.08335509,  0.56846498,  1.52185726,  0.11670154],
       [ 0.20052987,  0.76904412,  1.29483717, -0.4082825 , -0.26772465,
        -0.32687081,  0.38153842,  1.79767255,  0.15796644,  1.03330214],
       [-0.19690911,  1.05263158,  0.975908  , -0.67140204, -0.26772465,
        -0.32687081, -0.28954975, -0.6607426 , -0.1567776 , -0.97373513],
       [-0.26917075, -1.19611852,  1.06360535, -0.01360321, -0.26772465,
        -0.32687081,  1.45586509, -0.6607426 ,  0.47271047, -0.76194009],
       [-1.09114683,  0.50169215,  0.16329071,  1.69667375, -0.26772465,
        -0.32687081, -0.72640122, -0.6607426 , -1.20592439,  0.39597918]])

In [16]:
from sklearn.neural_network import MLPClassifier

In [17]:
mlpc = MLPClassifier().fit(X_train,y_train)

C:\Users\cakahraman\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [18]:
mlpc_params = {"alpha": [0.1, 0.01, 0.02, 0.005, 0.0001,0.00001],
              "hidden_layer_sizes": [(10,10,10),
                                     (100,100,100),
                                     (100,100),
                                     (3,5), 
                                     (5, 3)],
              "solver" : ["lbfgs","adam","sgd"],
              "activation": ["relu","logistic"]}

In [19]:
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, cv = 3, n_jobs = -1, verbose = 2)

mlpc_cv_model.fit(X_train_scaled, y_train)

#to save time cv=3

Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  6.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_sta...
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005, 0.0001, 1e-05],
               

In [20]:
print("En iyi parametreler:" + str(mlpc_cv_model.best_params_))

En iyi parametreler:{'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (100, 100, 100), 'solver': 'lbfgs'}


In [21]:
mlpc_tuned = MLPClassifier(activation = "relu",
                          alpha = 0.1,
                          hidden_layer_sizes = (100,100,100),
                          solver = "lbfgs")

In [22]:
mlpc_tuned.fit(X_train_scaled, y_train)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = mlpc_tuned.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.9488448844884488

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       554
           1       0.77      0.58      0.66        52

    accuracy                           0.95       606
   macro avg       0.87      0.78      0.82       606
weighted avg       0.94      0.95      0.95       606



In [ ]:
#SVM Support Vector Machines

In [25]:
from sklearn.svm import SVC

In [26]:
svm_model = SVC(kernel = "linear").fit(X_train[0:50], y_train[0:50])

In [27]:
y_pred = svm_model.predict(X_test)

In [28]:
svc_params = {"C": np.arange(1,10)}
svc = SVC(kernel = "linear")
svc_cv_model = GridSearchCV(svc,svc_params,
                           cv=2, 
                           n_jobs = -1,
                           verbose = 2)

svc_cv_model.fit(X_train[0:50],y_train[0:50])

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.5min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [29]:
print("En iyi parametreler:" + str(svc_cv_model.best_params_))

En iyi parametreler:{'C': 1}


In [33]:
svc_tuned = SVC(kernel = "linear", C = 1)

In [34]:
svc_tuned.fit(X_train_scaled, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [35]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = svc_tuned.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.9240924092409241

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       554
           1       1.00      0.12      0.21        52

    accuracy                           0.92       606
   macro avg       0.96      0.56      0.58       606
weighted avg       0.93      0.92      0.90       606

